<ul>
    <li> <h2><b>Universidade de Brasília - UnB</b></h2> </li>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Primeira (1/5) Atividade Prática Avaliativa
    Entrega da Atividade 01/10/2020 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol> 
<li> Colete os preços e volumes de um conjunto de ativos durante o período de 3 (três) anos, pertencentes a no mínimo 5 (cinco) Índices de Segmentos e Setoriais diferentes, e separe os 50 (cinquenta) ativos mais líquidos; </li><p></p>
    <li>Construa uma amostra de treinamento e outra de teste, sendo a primeira com os 2 (dois) primeiros anos e a segunda com o último anos; </li> <p></p>
    <li> Na amostra de treinamento, aplique uma clusterização não supervisionada (K-Means ou DBSCAN) e separe um ativo para cada clusters, com o critério que desejar (sugestão maior índice Sharpe); </li><p></p>
    <li> Aplique a Teoria do Portfolio Moderno de Markowitz nos ativos escolhidos no passo anterior, no sentido de alocar um capital de R$100.000,00; </li><p></p>
    <li> Verifique o resultado da alocação de capital com Markowitz, nos ativos separados na fase de treinamento, amostra de teste que representa o último ano da aquisição; </li><p></p>
    <li> Descreva suas impressores relativas ao processo e resultados encontrados. </li>
</ol>
</div>


### Questão 1: 

## Extração

Os dados das ações que compõem cada índice são extraídas diretamente da B3 através do seguinte [link](http://www.b3.com.br/pt_br/market-data-e-indices/indices/acoes-por-indice/), no fim da paǵina existe uma opção para realizar o download da planilha contendo as informações apresentadas.

In [27]:
import pandas as pd
import numpy as np

In [28]:
stock_by_index = pd.read_html('data/AcoesIndices.SetDez.2020.xls')[0]

In [29]:
stock_by_index.head()

RELA��O DE A��ES POR �NDICE - SETEMBRO A DEZEMBRO DE 2020Atualizado em 22/09/2020  \
                                                                            �ndices   
                                                                 Unnamed: 0_level_2   
0                                                 3M                                  
1                                         ABC BRASIL                                  
2                                       ADVANCED MIC                                  
3                                        ADVANCED-DH                                  
4                                        AES TIETE E                                  

                                                                       ...  \
                                                                       ...   
  Unnamed: 1_level_2    BDRX IBOV    IBRA IBXL IBXX ICO2 ICON    IDIV  ...   
0                DRN  MMMC34  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
1              PN N2     NaN  NaN   ABCB4  NaN  NaN  NaN  NaN   ABCB4  ...   
2                DRN  A1MD34  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
3                 ON     NaN  NaN     NaN  NaN  NaN  NaN  NaN     NaN  ...   
4             UNT N2     NaN  NaN  TIET11  NaN  NaN  NaN  NaN  TIET11  ...   

                                                                 
                                                                 
  IGNM IMAT IMOB INDX    ISEE    ITAG IVBX MLCX    SMLL    UTIL  
0  NaN  NaN  NaN  NaN     NaN     NaN  NaN  NaN     NaN     NaN  
1  NaN  NaN  NaN  NaN     NaN   ABCB4  NaN  NaN   ABCB4     NaN  
2  NaN  NaN  NaN  NaN     NaN     NaN  NaN  NaN     NaN     NaN  
3  NaN  NaN  NaN  NaN     NaN   ADHM3  NaN  NaN     NaN     NaN  
4  NaN  NaN  NaN  NaN  TIET11  TIET11  NaN  NaN  TIET11  TIET11  

[5 rows x 25 columns]

In [30]:
# Fix MultiIndex on columns
stock_by_index.columns = [col[2] for col in stock_by_index.columns]

In [31]:
stock_by_index.head()

,Unnamed: 0_level_2,Unnamed: 1_level_2,BDRX,IBOV,IBRA,IBXL,IBXX,ICO2,ICON,IDIV,...,IGNM,IMAT,IMOB,INDX,ISEE,ITAG,IVBX,MLCX,SMLL,UTIL
0,3M,DRN,MMMC34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABC BRASIL,PN N2,NaN,NaN,ABCB4,NaN,NaN,NaN,NaN,ABCB4,...,NaN,NaN,NaN,NaN,NaN,ABCB4,NaN,NaN,ABCB4,NaN
2,ADVANCED MIC,DRN,A1MD34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADVANCED-DH,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ADHM3,NaN,NaN,NaN,NaN
4,AES TIETE E,UNT N2,NaN,NaN,TIET11,NaN,NaN,NaN,NaN,TIET11,...,NaN,NaN,NaN,NaN,TIET11,TIET11,NaN,NaN,TIET11,TIET11


Remove as duas primeiras colunas que não são relevantes para a análise.

In [32]:
stock_by_index.drop(stock_by_index.columns[:2], axis=1, inplace=True)

## Selecionando os índices com o maior número de ativos

Foram selecionados os **5** primeiros índices que contém o maior número de ativos.

In [58]:
qtd_stocks_by_indexes = stock_by_index.iloc[-1].astype('int64')
selected_indexes = qtd_stocks_by_indexes.sort_values(ascending=False).head(5).index.tolist()

In [48]:
print('Índices selecionados: %s' % selected_indexes)

Índices selecionados: ['IGCX', 'ITAG', 'IBRA', 'IGCT', 'IGNM']


## Extraindo a lista de ativos que fazem parte dos índices

A seguir é feita a seleção no dataframe original das colunas correspondentes aos índices selecionados. É necessário descartar a última linha do dataframe original, pois essa linha não carrega informações relevantes para as etapas seguintes.

In [125]:
stocks_selected_indexes = stock_by_index[:-1][selected_indexes]

Para descartar os valores duplicados, primeiro é necessário substituir os valors `nan` por `'-'`. Em seguida aplicamos o método `numpy.unique` no dataframe resultante para ter como resultado somente as ações únicas através dos 5 índices. Por último removemos o valor `'-'`.

In [131]:
stock_list = np.unique(stocks_selected_indexes.fillna('-')).tolist()
stock_list.remove('-')

Como resultado temos uma lista de:

In [146]:
print("%s ativos únicos" % len(stock_list))

223 ativos únicos


## Baixando dados das cotações

In [185]:
import pandas_datareader as pdr
from tqdm import tqdm

In [186]:
def download_yahoo_quotes(list_symbols, start_date, final_date):
    """
    Realiza o download das cotações do Yahoo.
    
    Params:
        list_symbols : list 
            Lista de símbolos dos ativos escolhidos;
        start_date : str
            Data inicial do período desejado no formato: %Y-%m-%d
        final_date : str
            Data final do período desejado no formato: %Y-%m-%d
    """
    daily_quotes = pd.DataFrame()
    daily_volume = pd.DataFrame()
    symbols = []
    for symbol in tqdm(list_symbols):
        
        if not symbol.endswith('.SA'):
            symbol_sa = "%s.SA" % symbol
        else:
            symbol_sa = symbol
            
        try:
            df = pdr.get_data_yahoo(symbol_sa, start_date, final_date)

            close_price = df["Adj Close"]
            volume = df["Volume"]
            symbols.append(symbol)

            daily_quotes = pd.concat([daily_quotes, close_price], axis=1)
            daily_volume = pd.concat([daily_volume, volume], axis=1)

            daily_quotes.columns = symbols
            daily_volume.columns = symbols
        except:
            print('Erro(%s) ao realizar o download do ativo: %s' % (symbol_sa))
            
    return daily_quotes, daily_volume

In [ ]:
start_date = '2017-09-01'
final_date = '2020-09-01'

raw_stocks_quotes, stocks_volumes = download_yahoo_quotes(stock_list, start_date, final_date)

## Selecionando os ativos com maior liquidez

Os 50 ativos mais líquidos serão selecionados pelos que tem a maior média de volume no período analisado.

In [202]:
qtd_stocks = 50
high_volume_stocks_sym = stocks_volumes.mean().sort_values(ascending=False).head(qtd_stocks).index.tolist()

In [207]:
stock_quotes = raw_stocks_quotes[high_volume_stocks_sym].copy()

In [ ]:
stock_quotes.head()

### Questão 2: 

### Questão 3: 

### Questão 4: 

### Questão 5: 

### Questão 6: 